## 필요한 라이브러리 import

In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import glob

## 프로게이머 데이터 불러오기

In [2]:
team_inf = pd.read_csv("team_info.csv",engine="python")
team_inf = team_inf[["user_team","user_name","user_id"]]
team_inf[:5]

,user_team,user_name,user_id
0,ktrolster,score,1110551del
1,ktrolster,score,좋은날
2,ktrolster,score,열심히할게요오
3,ktrolster,mata,시어니보고싶움
4,ktrolster,smeb,smeb00


In [3]:
team_inf.to_csv("C:\\Users\\user\\Documents\\Python Workspace\\새벽 3시\\progamer 긁기\\data\\{0}_{1}_{2}.csv".format("a","b","감자"))

In [4]:
user_id = team_inf["user_id"]
user_team = team_inf["user_team"]
user_nick = team_inf["user_name"]
user_id[:10]
# for i in user_id:
#     print(i)
#     break

0    1110551del
1           좋은날
2       열심히할게요오
3       시어니보고싶움
4        smeb00
5          rrrr
6       케이티pawn
7    cyberbully
8    deft123456
9         똥좀싸겟다
Name: user_id, dtype: object

In [5]:
user_team[:5]
user_nick[:10]

0    score
1    score
2    score
3     mata
4     smeb
5     rush
6     pawn
7     rush
8     deft
9     pawn
Name: user_name, dtype: object

## Selenium에 필요한 크롬드라이버 주소와 크롤링 시작지점과 끝나는 지점 설정

In [13]:
# user_list2 = ['TOP아빠','Delicious Fish','슬프다고말하면','담원 너구리','IGS Coli','지현이의빗자루','담원 얼라이브','DWG BeryL'] ## Username
driver_address = 'D:\Downloads/chromedriver'
## 수정할것 2가지
start_number = 88 # 시작할 위치
end_number = 101

count = 1; # start number
match_number_start = 301256 # 시작할 match number , 충분히 크게 안 겹치게만

## 크롤링 코드

In [16]:
def match_data_load(user_name,user_team,user_nick, match_number_start):
    file_name = "C:\\Users\\user\\Documents\\Python Workspace\\새벽 3시\\progamer 긁기\\data\\{0}_{1}_{2}.csv".format(user_team,user_nick,user_name)
    # csv파일로 저장할 때 사용할 파일이름
    driver = webdriver.Chrome(driver_address) # 크롬드라이버 실행
    driver.implicitly_wait(3) # 대기
    driver.get('http://www.op.gg/summoner/userName={0}'.format(user_name)) # 크롤링을 할 주소 열기
    
    driver.find_element_by_xpath('//a[@data-type="soloranked"]').click() # 솔로랭크 클릭
    
    game_detail_button = driver.find_elements_by_css_selector('a.Button.MatchDetail') # 더보기 버튼 찾기
    for a in game_detail_button:
        a.click()
        time.sleep(1)
    time.sleep(3)    # 더보기 버튼 클릭하는 작업. 로딩을 기다리기 위해 time.sleep() 사용
    html = driver.page_source
    PlayerData = BeautifulSoup(html, 'html.parser') # html 주소 불러오기

    team_raw = PlayerData.find_all('tbody', {'class':'Content'}) # 게임에서 팀과 관련된 정보
    game_raw = PlayerData.find_all('thead', {'class':'Header'}) # 승패 관련 정보
    object_raw = PlayerData.find_all('div', {'class':'Summary'}) # 

    # 데이터를 저장할 빈 리스트
    win_lose = []
    champion_list = []
    name_list = []
    kill_list = []
    death_list = []
    assist_list = []
    damage_list = []
    buy_ward_list = []
    build_ward_list1 = []
    build_ward_list2 = []
    CS_list = []
    CSperMin_list = []
    object_list1 = []
    object_list2 = []
    object_list3 = []
    

    retry = 0
    index = []
    idx = 0
    
    # 게임 승패 정리
    for gline in game_raw:
        idx += 1
        game_result_raw = gline.find_all('span', {'class':'GameResult'})

        game_result = []
        for gresult in game_result_raw:
            gresult = gresult.text.strip()
            if (gresult != "승리") & (gresult != "패배"):
                retry += 1
                index.append(idx)
                continue
            game_result.append(gresult)
        if retry == 0:
            win_lose.append(game_result)
        else:
            retry -= 1
    #
    for obj in object_raw:
        idx = 0
        for i in obj.find_all('div',{'class':'ObjectScore'}):
            tmp = []
            if idx == 0:
                idx += 1
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list1.append(tmp)
            elif idx == 1:
                idx += 1
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list2.append(tmp)
            elif idx == 2:
                idx = 0
                for j in range(5):
                    tmp.append(i.text.strip())
                object_list3.append(tmp)    
    idx=0
    
    # 게임 결과 정보 크롤링 작업
    for line in team_raw:
        idx += 1
        if idx in index:
            continue
        teamdata = []
        # 각 해당하는 정보 html에서 찾는 작업
        team_champion_raw = line.find_all('td', {'class':'ChampionImage Cell'})
        team_name_raw = line.find_all('td', {'class':'SummonerName Cell'})
        team_kill_raw = line.find_all('span',{'class':'Kill'})
        team_death_raw = line.find_all('span',{'class':'Death'})
        team_assist_raw = line.find_all('span',{'class':'Assist'})
        team_damage_raw = line.find_all('div',{'class':'ChampionDamage'})
        team_buy_ward_raw = line.find_all('div',{'class':'Buy'})
        team_build_ward_raw = line.find_all('div',{'class':'Stats'})
        team_CS_raw = line.find_all('div',{'class':'CS'})
        team_CSperMin_raw = line.find_all('div',{'class':'CSPerMinute'})
        
        # 각 팀마다의 정보를 저장할 빈 리스트 설정
        team_champion = []
        # 각 팀의 챔피언 정보를 team_champion에 저장
        for champ in team_champion_raw:
            champ = champ.text.strip()
            champ = champ[:champ.find('\n')]
            team_champion.append(champ)
        champion_list.append(team_champion)
        # 이후 각각의 정보에 대해서 같은 방식으로 진행

        team_name = []
        for name in team_name_raw:
            name = name.text.strip()
            team_name.append(name)
        name_list.append(team_name)

        team_kill_participation = []

        team_kill = []
        for kills in team_kill_raw:
            kills = kills.text.strip()
            team_kill.append(kills)
        kill_list.append(team_kill)

        team_death = []
        for deaths in team_death_raw:
            deaths = deaths.text.strip()
            team_death.append(deaths)
        death_list.append(team_death)

        team_assist = []
        for assists in team_assist_raw:
            assists = assists.text.strip()
            team_assist.append(assists)
        assist_list.append(team_assist)

        team_damage = []
        for damage in team_damage_raw:
            damage = damage.text.strip().replace(",","")
            team_damage.append(damage)
        damage_list.append(team_damage)   

        team_buy_ward = []
        for buy_ward in team_buy_ward_raw:
            buy_ward = buy_ward.text.strip()
            team_buy_ward.append(buy_ward)
        buy_ward_list.append(team_buy_ward)  

        team_build_ward1 = []
        team_build_ward2 = []
        for build_ward in team_build_ward_raw:
            build_ward = build_ward.text.strip().replace("\t","").replace("\n","").replace("/","")
            team_build_ward1.append(build_ward[:build_ward.find(" ")])
            team_build_ward2.append(build_ward[build_ward.find(" "):].replace(" ",""))
        build_ward_list1.append(team_build_ward1)
        build_ward_list2.append(team_build_ward2)

        team_CS = []
        for CS in team_CS_raw:
            CS = CS.text.strip()
            team_CS.append(CS)
        CS_list.append(team_CS)  

        team_CSperMin = []
        for CSperMin in team_CSperMin_raw:
            CSperMin = CSperMin.text.strip()
            team_CSperMin.append(CSperMin[build_ward.find(" "):].replace(" ","").replace("당",""))
        CSperMin_list.append(team_CSperMin)  

    win_lose = win_lose[1:] # 첫 번째 항목은 관련없는 빈 리스트라서 삭제

    KDAsorted = [] # 데이터를 최종적으로 정리할 리스트
    matchorder = match_number_start # 각각 경기에 대한 인덱스
    matchnumber = 1 # 정보처리를 위한 인덱스
    for order in range(int((len(win_lose)+1)/2)): # 각각 경기에 대해 승패가 생기므로 N개의 경기라면 win_lose는 2N개의 자료이므로 2로 나눈다.
        matchnumber = matchnumber-1
        # 한 경기당 10명이 플레이하고 5명씩 팀을 이루므로 5개씩 2개의 for문을 사용했다.
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            team_member.append(int(damage_list[matchnumber][i]))
            team_member.append(int(buy_ward_list[matchnumber][i]))
            team_member.append(int(build_ward_list1[matchnumber][i]))
            team_member.append(int(build_ward_list2[matchnumber][i]))
            team_member.append(int(CS_list[matchnumber][i]))
            team_member.append(float(CSperMin_list[matchnumber][i]))
            team_member.append(int(object_list1[matchnumber][i]))
            team_member.append(int(object_list2[matchnumber][i]))
            team_member.append(int(object_list3[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 1
        for i in range(5):
            team_member = []
            team_member.append(matchorder)
            team_member.append(win_lose[matchnumber][0])
            team_member.append(champion_list[matchnumber][i])
            team_member.append(name_list[matchnumber][i])
            team_member.append(int(kill_list[matchnumber][i]))
            team_member.append(int(death_list[matchnumber][i]))
            team_member.append(int(assist_list[matchnumber][i]))
            team_member.append(int(damage_list[matchnumber][i]))
            team_member.append(int(buy_ward_list[matchnumber][i]))
            team_member.append(int(build_ward_list1[matchnumber][i]))
            team_member.append(int(build_ward_list2[matchnumber][i]))
            team_member.append(int(CS_list[matchnumber][i]))
            team_member.append(float(CSperMin_list[matchnumber][i]))
            team_member.append(int(object_list1[matchnumber][i]))
            team_member.append(int(object_list2[matchnumber][i]))
            team_member.append(int(object_list3[matchnumber][i]))
            KDAsorted.append(team_member)
        matchnumber += 2
        matchorder += 1
    # KDAsorted를 정보처리를 위해 DataFrame으로 변환
    kda_data = pd.DataFrame(KDAsorted, columns=['MatchNumber','GameResult', 'Champion', 'Nickname', 
                                                'Kill', 'Death', 'Assist', 'Damage','Buy_Pward','Build_ward','Destroy_ward',
                                                'CS','Baron','Dragon','Tower'])
    # kda_data를 csv파일로 저장
    kda_data.to_csv(file_name)
    
    # 크롤링이 끝난 driver창을 끈다.
    driver.quit()
    return matchorder

## 진행상황 확인용 코드

In [14]:
for user_name,u_t,u_nk in zip(user_id,user_team,user_nick):
    if count == end_number+1:
        break
    if count < start_number:
        count += 1
        continue
    match_number_start = match_data_load(user_name,u_t,u_nk, match_number_start)
    print("complete! count :",count,", error_start : count",count+1,", start_number", match_number_start)
    count += 1

complete! count : 88 , error_start : count 89 , start_number 301256
complete! count : 89 , error_start : count 90 , start_number 301276
complete! count : 90 , error_start : count 91 , start_number 301276
complete! count : 91 , error_start : count 92 , start_number 301296
complete! count : 92 , error_start : count 93 , start_number 301316
complete! count : 93 , error_start : count 94 , start_number 301336
complete! count : 94 , error_start : count 95 , start_number 301356
complete! count : 95 , error_start : count 96 , start_number 301362
complete! count : 96 , error_start : count 97 , start_number 301382
complete! count : 97 , error_start : count 98 , start_number 301402
complete! count : 98 , error_start : count 99 , start_number 301422
complete! count : 99 , error_start : count 100 , start_number 301442
complete! count : 100 , error_start : count 101 , start_number 301461
complete! count : 101 , error_start : count 102 , start_number 301480
